In [7]:
import pandas as pd
import numpy as np

In [9]:
# Se carga el dataframe
df_homicidios = pd.read_csv(r"Datasets\homicidios_clean.csv")
# Función que selecciona los meses correspondientes a cada semestre
def Semestre(mes):
    primer_semestre = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio']
    return 1 if mes.lower() in primer_semestre else 2

# Crea la columna semestre, y les asigna el valor correspondiente segun el mes, para poder trabajarlo al KPI
df_homicidios['Semestre'] = df_homicidios['Mes'].apply(Semestre)
df_homicidios.head()

,Id,Nro víctimas,Año,Mes,Día,Hora estandar,Tipo de calle,Cruce,Lugar del hecho,Comuna,...,Pos y,Participantes,Víctima,Acusado,Hora,Rol,Sexo,Edad,FECHA_FALLECIMIENTO,Semestre
0,2016-0001,1,2016,Enero,Viernes,4,AVENIDA,SI,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,...,-34.687570,MOTO-AUTO,MOTO,AUTO,4,CONDUCTOR,MASCULINO,19,1/1/2016,1
1,2016-0002,1,2016,Enero,Sábado,1,GRAL PAZ,SI,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,...,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,1,CONDUCTOR,MASCULINO,70,1/2/2016,1
2,2016-0003,1,2016,Enero,Domingo,7,AVENIDA,NO,ENTRE RIOS AV. 2034,1,...,-34.631894,MOTO-AUTO,MOTO,AUTO,7,CONDUCTOR,MASCULINO,30,1/3/2016,1
3,2016-0004,1,2016,Enero,Domingo,0,AVENIDA,SI,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,...,-34.680930,MOTO-SD,MOTO,SD,0,CONDUCTOR,MASCULINO,18,SD,1
4,2016-0005,1,2016,Enero,Jueves,5,AVENIDA,SI,"SAN JUAN AV. y SAENZ PEÑA, LUIS, PRES.",1,...,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,5,CONDUCTOR,MASCULINO,29,2/1/2016,1


In [20]:
# Datos de poblacion por año
data = {
    'Año': [2016, 2017, 2018, 2019, 2020, 2021],
    'Población': [3059122, 3063728, 3068043, 3072029, 3075646, 3078836]
}

# Crear el DataFrame
df_pob = pd.DataFrame(data)

# Mostrar el DataFrame
print(df_pob)

df_pob.to_csv(r"Datasets\Poblacion.csv", index=False)

    Año  Población
0  2016    3059122
1  2017    3063728
2  2018    3068043
3  2019    3072029
4  2020    3075646
5  2021    3078836


1. Reducir un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA

In [15]:
#Mergeamos Dfs
df_merge = pd.merge(df_homicidios,df_pob, on = 'Año')
df_pobKPI = df_merge.groupby(['Año', 'Semestre', 'Población'])['Nro víctimas'].sum().reset_index()
df_pobKPI['Tasa'] = round( (df_pobKPI['Nro víctimas'] / df_pobKPI['Población']) *100000 ,2 )
df_pobKPI['TasaAnterior'] = round( df_pobKPI['Tasa'].shift(periods=1, fill_value=0) ,2 )
df_pobKPI['Cambio'] = round(( df_pobKPI['Tasa'] - df_pobKPI['TasaAnterior'] ) / df_pobKPI['TasaAnterior'] *100 ,2 ) 
df_pobKPI['Cambio'] = df_pobKPI['Cambio'].replace(np.inf, 0)    # The innfinite is replaced by 0
df_pobKPI

,Año,Semestre,Población,Nro víctimas,Tasa,TasaAnterior,Cambio
0,2016,1,3059122,63,2.06,0.00,0.00
1,2016,2,3059122,79,2.58,2.06,25.24
2,2017,1,3063728,60,1.96,2.58,-24.03
3,2017,2,3063728,63,2.06,1.96,5.10
4,2018,1,3068043,64,2.09,2.06,1.46
5,2018,2,3068043,73,2.38,2.09,13.88
6,2019,1,3072029,55,1.79,2.38,-24.79
7,2019,2,3072029,47,1.53,1.79,-14.53
8,2020,1,3075646,31,1.01,1.53,-33.99
9,2020,2,3075646,44,1.43,1.01,41.58


2. Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

In [16]:
# Filtra el DataFrame para incluir solo las filas donde la víctima sea una moto
df_moto = df_merge[df_merge['Víctima'] == 'MOTO']

# Agrupa por año y cuenta el número de accidentes únicos
df_moto = df_moto.groupby(['Año'])['Id'].nunique().reset_index()
df_moto = df_moto.rename(columns={'Id': 'Acc_fatal_moto'})

# Calcula la evolución de los accidentes mortales con víctimas en moto
df_moto['Evol_año_anterior_m'] = round(df_moto['Acc_fatal_moto'].pct_change() * 100, 2)

# Reemplaza los valores NaN con 0 en la primera fila
df_moto['Evol_año_anterior_m'] = df_moto['Evol_año_anterior_m'].fillna(0)

# Agregar la columna "Reducción" con los valores opuestos de "Evol_año_anterior"
df_moto['Reducción_m'] = -df_moto['Evol_año_anterior_m']

# Imprime el DataFrame resultante
df_moto

,Año,Acc_fatal_moto,Evol_año_anterior_m,Reducción_m
0,2016,68,0.00,-0.00
1,2017,54,-20.59,20.59
2,2018,53,-1.85,1.85
3,2019,50,-5.66,5.66
4,2020,26,-48.00,48.00
5,2021,46,76.92,-76.92


3. Reducir en un 10% la cantidad de accidentes mortales de peatones en el último año, en CABA, respecto al año anterior.

In [17]:
# Filtra el DataFrame para incluir solo las filas donde la víctima sea una moto
df_peaton = df_merge[df_merge['Víctima'] == 'PEATON']

# Agrupa por año y cuenta el número de accidentes únicos
df_peaton = df_peaton.groupby(['Año'])['Id'].nunique().reset_index()
df_peaton = df_peaton.rename(columns={'Id': 'Acc_fatal_peaton'})

# Calcula la evolución de los accidentes mortales con víctimas en moto
df_peaton['Evol_año_anterior_p'] = round(df_peaton['Acc_fatal_peaton'].pct_change() * 100, 2)

# Reemplaza los valores NaN con 0 en la primera fila
df_peaton['Evol_año_anterior_p'] = df_peaton['Evol_año_anterior_p'].fillna(0)

# Agregar la columna "Reducción" con los valores opuestos de "Evol_año_anterior"
df_peaton['Reducción_p'] = -df_peaton['Evol_año_anterior_p']

# Imprime el DataFrame resultante
df_peaton

,Año,Acc_fatal_peaton,Evol_año_anterior_p,Reducción_p
0,2016,49,0.00,-0.00
1,2017,49,0.00,-0.00
2,2018,64,30.61,-30.61
3,2019,33,-48.44,48.44
4,2020,34,3.03,-3.03
5,2021,33,-2.94,2.94


In [18]:
#Mergeamos los Dfs
df_KPI = df_pobKPI.merge(df_moto, on='Año', how='left')
df_KPI = df_KPI.merge(df_peaton, on='Año', how='left')
df_KPI

,Año,Semestre,Población,Nro víctimas,Tasa,TasaAnterior,Cambio,Acc_fatal_moto,Evol_año_anterior_m,Reducción_m,Acc_fatal_peaton,Evol_año_anterior_p,Reducción_p
0,2016,1,3059122,63,2.06,0.00,0.00,68,0.00,-0.00,49,0.00,-0.00
1,2016,2,3059122,79,2.58,2.06,25.24,68,0.00,-0.00,49,0.00,-0.00
2,2017,1,3063728,60,1.96,2.58,-24.03,54,-20.59,20.59,49,0.00,-0.00
3,2017,2,3063728,63,2.06,1.96,5.10,54,-20.59,20.59,49,0.00,-0.00
4,2018,1,3068043,64,2.09,2.06,1.46,53,-1.85,1.85,64,30.61,-30.61
5,2018,2,3068043,73,2.38,2.09,13.88,53,-1.85,1.85,64,30.61,-30.61
6,2019,1,3072029,55,1.79,2.38,-24.79,50,-5.66,5.66,33,-48.44,48.44
7,2019,2,3072029,47,1.53,1.79,-14.53,50,-5.66,5.66,33,-48.44,48.44
8,2020,1,3075646,31,1.01,1.53,-33.99,26,-48.00,48.00,34,3.03,-3.03
9,2020,2,3075646,44,1.43,1.01,41.58,26,-48.00,48.00,34,3.03,-3.03


In [19]:
df_homicidios.to_csv(r"Datasets\KPIs.csv", index=False)